### 제 2유형 연습하기. 타이타닉 생존자 분류하기 풀이

#### 1. 라이브러리 및 데이터 확인 
#### 2. 데이터 탐색(EDA)
#### 3. 데이터 전처리 및 분리 (결측치, 이상치 대체)
#### 4. 모델링 및 성능평가
#### 5. 예측값 제출

In [ ]:
# 라이브러리 및 데이터 확인 
import pandas as pd 
import numpy as np  